## Displacement

In [164]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from model.Displacement.regression import CNN, CNN_FineTune
from dataset.Displacement.DamageIdentification import DamageIdentificationDataset
from torch.utils.data import DataLoader

In [174]:
mode = "test"
dataset = DamageIdentificationDataset(path="./Data", mode=mode)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [175]:
scratch = CNN.load_from_checkpoint("./Logs/Identification/Displacement-regression/From_scratch/version_2/checkpoints/epoch=00343-val_loss=0.0000.ckpt").eval().to("cuda")
transfer_AE = CNN.load_from_checkpoint("./Logs/Identification/Displacement-regression/Transfer_AE/version_2/checkpoints/epoch=00034-val_loss=0.0197.ckpt").eval().to("cuda")
transfer_DamageAE = CNN.load_from_checkpoint("./Logs/Identification/Displacement-regression/Transfer_DamageAE/version_2/checkpoints/epoch=00500-val_loss=0.0142.ckpt").eval().to("cuda")
transfer_TripletAE = CNN.load_from_checkpoint("./Logs/Identification/Displacement-regression/Transfer_TripletAE/version_2/checkpoints/epoch=00497-val_loss=0.0122.ckpt").eval().to("cuda")

In [176]:
scratch_prediction = []


target_list = []
situation_list = []

loss = 0
criterion = nn.MSELoss()

for input, target, situation in dataloader:
    with torch.no_grad():
        input = input.to("cuda") 
        target = target.to("cuda")      
        pred = scratch(input)
        scratch_prediction += pred.unsqueeze(0)

        loss += criterion(pred, target)

        target_list += target.unsqueeze(0)
        situation_list += situation.unsqueeze(0)

print("Average MSE Loss: ", (loss/len(dataloader)).cpu().numpy())

scratch_prediction = torch.concat(scratch_prediction).cpu().numpy()

target_list = torch.concat(target_list).cpu().numpy()
situation_list = torch.concat(situation_list).cpu().numpy()

Average MSE Loss:  0.006623704


In [177]:
def check_unique_target(target_list):
    unique_target_list = []
    for i in target_list:
        add = True
        for j in unique_target_list:
            if (i == j).all():
                add = False
                break
        if add:
            unique_target_list.append(i)
    return unique_target_list

In [178]:
def get_index(target, target_list):
    index = []
    for i in target_list:
        if (target == i).all():
            index.append(True)
        else:
            index.append(False)

    return np.array(index)

In [179]:
def predition_csv(pred_list, target_list, model_name, mode):
    unique_target_list = check_unique_target(target_list)
    for target in unique_target_list:
        index = get_index(target, target_list)
        
        pred = pred_list[index]
        pred_no7, pred_no22, pred_no38 = pred[:, 0], pred[:, 1], pred[:, 2]
        target_no7, target_no22, target_no38 = target_list[index][:, 0], target_list[index][:, 1], target_list[index][:, 2]
        df = pd.DataFrame({
            "Predition (No.7)":pred_no7,
            "Target (No.7)":target_no7,
            "Predition (No.22)":pred_no22,
            "Target (No.22)":target_no22,
            "Predition (No.38)":pred_no38,
            "Target (No.38)":target_no38,
        })
        if not os.path.exists(f"./prediction/{model_name}/{mode}"):
            os.makedirs(f"./prediction/{model_name}/{mode}")
        df.to_csv(f"./prediction/{model_name}/{mode}/Prediction_{int(target_no7[0]*100):02d}_{int(target_no22[0]*100):02d}_{int(target_no38[0]*100):02d}.csv", index=False)

In [180]:
from scipy.stats import norm

def draw_predition_distribution(pred_list, target_list, model_name, mode, precision=0.01):
    unique_target_list = check_unique_target(target_list)
    for target in unique_target_list:
        index = get_index(target, target_list)
        
        pred = pred_list[index]
        max_value, min_value = pred.max(), pred.min()
        pred_no7, pred_no22, pred_no38 = pred[:, 0], pred[:, 1], pred[:, 2]
        target_no7, target_no22, target_no38 = target_list[index][:, 0], target_list[index][:, 1], target_list[index][:, 2]
        df = pd.DataFrame({
            "Predition (No.7)":pred_no7,
            "Target (No.7)":target_no7,
            "Predition (No.22)":pred_no22,
            "Target (No.22)":target_no22,
            "Predition (No.38)":pred_no38,
            "Target (No.38)":target_no38,
        })
        if not os.path.exists(f"./prediction/{model_name}/{mode}"):
            os.makedirs(f"./prediction/{model_name}/{mode}")
        df.to_csv(f"./prediction/{model_name}/{mode}/Prediction_{int(target_no7[0]*100):02d}_{int(target_no22[0]*100):02d}_{int(target_no38[0]*100):02d}.csv", index=False)

        # plt.rcParams['text.usetex'] = True
        # plt.rcParams['text.latex.preamble'] = r'\centering'
        # (mu_no7, sigma_no7) = norm.fit(pred_no7)
        # (mu_no22, sigma_no22) = norm.fit(pred_no22)
        # (mu_no38, sigma_no38) = norm.fit(pred_no38)

        # fig, ax = plt.subplots(figsize=(6, 6))
        # bin_size = np.arange(0, 0.51, precision)

        # ax.hist(pred_no7, align="left", bins=bin_size, density=False, range=(min_value, max_value), facecolor='tab:blue', alpha=0.5)
        # ax.hist(pred_no22, align="left", bins=bin_size, density=False, range=(min_value, max_value), facecolor='tab:orange', alpha=0.5)
        # ax.hist(pred_no38, align="left", bins=bin_size, density=False, range=(min_value, max_value), facecolor='tab:green', alpha=0.5)

        # ax.axvline(x = target_no7, linestyle = "-", color = 'tab:blue', label = 'Target of No.7')
        # ax.axvline(x = target_no22, linestyle = "--", color = 'tab:orange', label = 'Target of No.22')
        # ax.axvline(x = target_no38, linestyle = "-.", color = 'tab:green', label = 'Target of No.38')

        # xmin, xmax, ymin, ymax = plt.axis()
        # x = np.linspace(xmin, xmax, 100)
        # y = norm.pdf(x, mu_no7, sigma_no7)
        # l = ax.plot(x, y, linestyle = 'dotted', color = "tab:blue")
        # y = norm.pdf(x, mu_no22, sigma_no22)
        # l = ax.plot(x, y, linestyle = 'dotted', color = "tab:orange")
        # y = norm.pdf(x, mu_no38, sigma_no38)
        # l = ax.plot(x, y, linestyle = 'dotted', color = "tab:green")
        # ax.set_title(
        #     r'$\mu_{no.07}=%.3f$, $\sigma_{no.07}=%.3f;$'
        #     r'$\mu_{no.22}=%.3f$, $\sigma_{no.22}=%.3f;$\\' 
        #     r'$\mu_{no.38}=%.3f$, $\sigma_{no.38}=%.3f;$' % (mu_no7, sigma_no7, mu_no22, sigma_no22, mu_no38, sigma_no38))
        # ax.set_xlabel("Reduction of elastic modulus")
        # fig.suptitle(r"$Target_{no.07}=%.2f $ $Target_{no.22}=%.2f$ $Target_{no.38}=%.2f $" %(target_no7, target_no22, target_no38), fontsize=14)
        # fig.legend(ncol = 3, bbox_to_anchor=(0.92, 0.04))
        # if not os.path.exists(f"./pic/distribution/{model_name}/{mode}"):
        #     os.makedirs(f"./pic/distribution/{model_name}/{mode}")
        # plt.savefig(f"./pic/distribution/{model_name}/{mode}/Scenario{int(target_no7*100)}_{int(target_no22*100)}_{int(target_no38*100)}.svg", bbox_inches='tight')
        # plt.savefig(f"./pic/distribution/{model_name}/{mode}/Scenario{int(target_no7*100)}_{int(target_no22*100)}_{int(target_no38*100)}.png", bbox_inches='tight')
        # plt.close()

    # if (mu_no7 - 3 * sigma_no7) <= target_no7 <= (mu_no7 + 3 * sigma_no7) and (mu_no22 - 3 * sigma_no22) <= target_no22 <= (mu_no22 + 3 * sigma_no22) and (mu_no38 - 3 * sigma_no38) <= target_no38 <= mu_no38 + 3 * sigma_no38:
    #     return True
    # else:
    #     return False

In [181]:
predition_csv(scratch_prediction, target_list, model_name="Scratch", mode=mode)

In [182]:
from dataset.Displacement.SyntheticFinetune import SyntheticDataset

In [183]:
def get_all_condition():
    all_condition = []
    for d7 in range(6):
        for d22 in range(6):
            for d38 in range(6):
                onehot7 = np.eye(6)[d7]
                onehot22 = np.eye(6)[d22]
                onehot38 = np.eye(6)[d38]
                all_condition.append((onehot7, onehot22, onehot38))
    return all_condition

In [196]:
mode = "valid"
dataset = SyntheticDataset(n_times=50, gan_checkpoint="./Logs/Generation/Displacement_WCGAN_GP/LAST/version_0/checkpoints/epoch=00196-fjd=0.0000.ckpt", \
                           defined_condition=get_all_condition(), real_involve=True, mode=mode)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [197]:
fine_tune = CNN_FineTune.load_from_checkpoint("./Logs/Identification/Displacement(FineTune)-regression/Involve_real_data/version_0/checkpoints/epoch=00387-val_loss=0.0000.ckpt").eval().to("cuda")

In [198]:
fine_tune_prediction = []

target_list = []

loss = 0
criterion = nn.MSELoss()
for input, target in dataloader:
    with torch.no_grad():
        input = input.to("cuda")    
        target = target.to("cuda")
        pred =  fine_tune(input)
        fine_tune_prediction += pred.unsqueeze(0)
        loss += criterion(target, pred)
        print(pred)
        print(target)
        print("==================================================")

        target_list += target.unsqueeze(0)

print("Average MSE Loss: ", (loss/len(dataloader)).cpu().numpy())

fine_tune_prediction = torch.concat(fine_tune_prediction).cpu().numpy()
target_list = torch.concat(target_list).cpu().numpy()

tensor([[ 0.2523,  0.0097,  0.0572],
        [ 0.2173,  0.0006,  0.0530],
        [ 0.2211,  0.0033,  0.0507],
        [ 0.2183,  0.0025,  0.0528],
        [ 0.1914, -0.0051,  0.0600],
        [ 0.2278,  0.0216,  0.0495],
        [ 0.1624,  0.0054,  0.0223],
        [ 0.1336,  0.0078,  0.0187],
        [ 0.1559,  0.0067,  0.0039],
        [ 0.1322,  0.0089,  0.0112],
        [ 0.1260,  0.0087,  0.0044],
        [ 0.1636, -0.0017,  0.0225],
        [ 0.1779,  0.0045,  0.0239],
        [ 0.1755,  0.0029,  0.0047],
        [ 0.1426,  0.0048,  0.0055],
        [ 0.1622,  0.0076,  0.0063],
        [ 0.1738,  0.0007,  0.0086],
        [ 0.1866,  0.0020,  0.0238],
        [ 0.1559,  0.0065,  0.0198],
        [ 0.1296, -0.0018,  0.0042],
        [ 0.1203,  0.0020,  0.0051],
        [ 0.1138,  0.0031,  0.0069],
        [ 0.1346,  0.0005,  0.0081],
        [ 0.1671,  0.0060,  0.0096],
        [ 0.1366, -0.0038,  0.0145],
        [ 0.1416, -0.0019,  0.0148],
        [ 0.1360, -0.0030,  0.0118],
 

In [195]:
unique_target_list = []
for i in target_list:
    add = True
    for j in unique_target_list:
        if (i == j).all():
            add = False
            break
    if add:
        unique_target_list.append(i)
    

In [132]:
len(unique_target_list)

216

In [133]:
target_list.shape

(5210, 3)

In [163]:
predition_csv(fine_tune_prediction, target_list, model_name="Scratch(synthetic-augment)", mode=mode)